<a href="https://colab.research.google.com/github/zhong338/pingzhong/blob/main/MultifactorModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']= ['SimHei']

import datetime
import statsmodels.api as sm 
from scipy import stats

In [ ]:
df_all = pd. read_excel('月度数据_1.2.0.xlsx',header = 1,index_col = 0)
df_stock = pd.read_excel('A股成分.xlsx')
date = df_all.columns

In [ ]:
df_all[:3]

,2012-01-31 00:00:00,2012-02-29 00:00:00,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,1.162892,1.195039,1.097898,1.104502,1.046476,1.011127,0.969181,0.910774,0.842739,0.824993,...,1.000162,1.046423,0.920248,1.041135,1.041614,1.043825,1.128369,1.176862,1.106656,1.169181
000002.SZ,1.588009,1.718787,1.718787,1.816410,1.871024,1.802251,1.841854,1.600397,1.682213,1.624242,...,2.177156,2.071742,1.915357,1.994984,2.018730,1.810961,1.749978,1.792545,1.871598,1.933989
000004.SZ,9.218794,10.511878,9.062732,9.507560,10.466119,9.440684,8.989553,8.944492,9.147264,9.225603,...,16.659146,18.526989,15.756735,15.551531,14.476452,15.525016,15.899769,15.383872,16.838532,1.364990


In [ ]:
df_stock[:3]

,Code,名称,行业类别,行业代码
0,000001.SZ,平安银行,银行,S480000
1,000002.SZ,万科A,房地产,S430000
2,000004.SZ,国华网安,医药生物,S370000


In [ ]:
#换手率数据
Turnover = pd.read_excel('月度数据_1.2.0.xlsx', header = 1,index_col = 0, sheet_name = '换手率')
Turnover[:3]

,2012-01-31 00:00:00,2012-02-29 00:00:00,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,5.9227,15.1235,16.6545,11.3631,11.4935,12.8000,8.8032,9.2906,9.7387,6.9194,...,15.5577,15.1028,10.8616,9.6805,9.6233,17.4331,13.7180,10.4053,8.4517,7.8140
000002.SZ,10.1242,17.4246,12.0537,8.9494,11.4631,10.8400,13.0062,12.0015,10.5298,8.1784,...,16.3671,13.2516,6.7084,6.4802,9.8128,8.9136,9.9530,11.9469,12.2058,19.4650
000004.SZ,29.6126,54.1288,49.7603,27.0772,32.1800,49.6159,207.0672,56.9107,41.9219,22.2088,...,26.6422,37.6220,34.4733,26.0555,22.0360,19.7063,36.3557,13.3637,27.4346,37.2601


In [ ]:
#成交量数据 & 流通股本数据
Volume = pd.read_excel('月度数据_1.2.0.xlsx', sheet_name = '成交量', header =1, index_col = 0)
CapitalStock = pd.read_excel('月度数据_1.2.0.xlsx', sheet_name = '流通股本', header =1, index_col = 0)
Volume[:3]

,2012-01-31 00:00:00,2012-02-29 00:00:00,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,183921042.0,4.696358e+08,5.171779e+08,352860001.0,3.569099e+08,3.974864e+08,2.733712e+08,2.885086e+08,3.024194e+08,214870356.0,...,2.671303e+09,2.593170e+09,1.864961e+09,1.662158e+09,1.652336e+09,2.993290e+09,2.538655e+09,2.019234e+09,1.640143e+09,1.516355e+09
000002.SZ,978179964.0,1.683528e+09,1.164602e+09,864667366.0,1.107535e+09,1.047364e+09,1.256639e+09,1.159596e+09,1.017391e+09,790201889.0,...,1.590086e+09,1.287407e+09,6.517398e+08,6.295582e+08,9.533243e+08,8.659557e+08,9.669779e+08,1.160658e+09,1.185824e+09,1.891029e+09
000004.SZ,24837662.0,4.540058e+07,4.173676e+07,22711235.0,2.699105e+07,4.161562e+07,1.736786e+08,4.773399e+07,3.516226e+07,18627774.0,...,2.209531e+07,3.120119e+07,2.858977e+07,2.160858e+07,1.820276e+07,1.627824e+07,3.003140e+07,1.103910e+07,2.266200e+07,3.101480e+07


In [ ]:
CapitalStock[:3]

,2012-01-31 00:00:00,2012-02-29 00:00:00,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,3.105359e+09,...,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.717025e+10,1.940575e+10,1.940575e+10,1.940575e+10,1.940575e+10
000002.SZ,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.661828e+09,9.662036e+09,9.662036e+09,9.662036e+09,9.662036e+09,...,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09,9.715170e+09
000004.SZ,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,8.387546e+07,...,8.293361e+07,8.293361e+07,8.293361e+07,8.293361e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.260458e+07,8.392068e+07


In [ ]:
#准备三个月换手率 sto_3m
sto_3m = pd.DataFrame()

for i in range (2,len(date)):
    sto_3m[date[i]] = (Volume[date[i]]+Volume[date[i-1]]
                                 +Volume[date[i-2]])/CapitalStock[date[i]]
sto_3m[:3]

,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,2012-11-30 00:00:00,2012-12-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.377005,0.431407,0.395107,0.356563,0.330966,0.308939,0.278325,0.259486,0.240414,0.368488,...,0.372685,0.417929,0.415220,0.356447,0.301653,0.367367,0.370214,0.389121,0.319391,0.266711
000002.SZ,0.396023,0.384275,0.324660,0.312525,0.353087,0.358475,0.355373,0.307098,0.285657,0.338312,...,0.341157,0.402909,0.363270,0.264401,0.230014,0.252063,0.286795,0.308136,0.341060,0.436175
000004.SZ,1.335015,1.309663,1.090176,1.088732,2.888631,3.135938,3.058998,1.210414,0.822744,0.627093,...,0.468050,0.777644,0.987371,0.981502,0.828055,0.679013,0.780978,0.694256,0.771537,0.771156


In [ ]:
#准备六个月换手率 sto_6m
sto_6m = pd.DataFrame()

for i in range (5,len(date)):
    k=5
    s=0
    while k>=0:
        s=s+Volume[date[i-k]]
        k=k-1
    sto_6m[date[i]]=s/Volume[date[i]]
sto_6m[:3]

,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,2012-11-30 00:00:00,2012-12-31 00:00:00,2013-01-31 00:00:00,2013-02-28 00:00:00,2013-03-29 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,5.730991,8.660170,7.577985,6.519275,8.533359,7.440371,2.868855,2.333910,3.807818,3.027048,...,4.665718,4.734191,6.464776,7.532003,7.477536,4.489113,5.240795,6.304686,7.624834,8.151134
000002.SZ,6.536293,5.669358,5.691984,6.342885,8.072275,6.534423,4.391612,4.688463,4.587558,4.017847,...,3.899935,4.937939,9.296884,9.344808,6.450008,6.903437,5.537834,4.504525,4.859320,3.714258
000004.SZ,4.885014,2.027503,7.425887,9.893925,18.456811,21.818607,16.487548,3.311956,5.042040,6.024668,...,3.163289,3.121810,3.801793,5.563375,7.300758,8.476090,4.858646,11.391311,5.287357,4.166666


In [ ]:
#涨跌幅数据
ret = pd.read_excel('月度数据_1.2.0.xlsx', sheet_name = '涨跌幅', header =1, index_col = 0)
ret[:3]

,2012-01-31 00:00:00,2012-02-29 00:00:00,2012-03-30 00:00:00,2012-04-27 00:00:00,2012-05-31 00:00:00,2012-06-29 00:00:00,2012-07-31 00:00:00,2012-08-31 00:00:00,2012-09-28 00:00:00,2012-10-31 00:00:00,...,2019-03-29 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,6.735087,2.764423,-8.128655,5.410567,-5.253623,-3.377948,-0.395778,-6.026490,-7.470049,0.975900,...,3.721683,8.034321,-12.057762,14.328093,2.539913,0.212314,10.098870,4.297627,-5.965560,7.586658
000002.SZ,2.409639,8.235294,0.000000,8.454106,3.006682,-3.675676,5.091124,-13.109426,5.112219,-1.304864,...,9.753483,-5.989583,-7.548476,4.157303,3.416037,-6.680365,0.387597,2.432432,4.410102,16.173285
000004.SZ,2.224969,14.026602,-13.785790,4.920049,10.082063,-9.797657,-5.785124,-0.501253,2.267003,1.354680,...,11.413882,16.658976,-14.952532,-1.302326,-12.818096,7.243243,-5.241935,-3.244681,9.455745,13.460573


In [ ]:
def mad(x,n):
    dm=x.median()
    dmad=((x-dm).abs()).median()
    max=dm+n*dmad
    min=dm-n*dmad
    return np.clip(x,min,max)

In [ ]:
sto_lm_s = pd.DataFrame()
for i in range(0,len(date)):
    sto_lm_s[date[i]]=mad(Turnover[date[i]],5)

In [ ]:
sto_3m_s = pd.DataFrame()
for i in range(2,len(date)):
    sto_3m_s[date[i]]=mad(sto_3m[date[i]],5)
sto_6m_s = pd.DataFrame()
for i in range(5,len(date)):
    sto_6m_s[date[i]]=mad(sto_6m[date[i]],5)

In [ ]:
ret_s = pd.DataFrame()
for i in range(0,len(date)):
    ret_s[date[i]]=mad(ret[date[i]],5)

In [ ]:
def standard(x):
    sigma=x.std()
    u=x.mean()
    return (x-u)/sigma

In [ ]:
for i in range(0,len(date)):
    sto_lm_s[date[i]]=standard(sto_lm_s[date[i]])
    
for i in range(2,len(date)):
    sto_3m_s[date[i]]=standard(sto_3m_s[date[i]])
    
for i in range(5,len(date)):
    sto_6m_s[date[i]]=standard(sto_6m_s[date[i]])
    
for i in range(0,len(date)):
    ret_s[date[i]]=standard(ret_s[date[i]])

In [ ]:
df_stock[:3]

,Code,名称,行业类别,行业代码
0,000001.SZ,平安银行,银行,S480000
1,000002.SZ,万科A,房地产,S430000
2,000004.SZ,国华网安,医药生物,S370000


In [ ]:
test = pd.DataFrame(ret_s[date[1]])
test.columns = ['涨跌幅']

In [ ]:
test['换手率']=sto_lm_s[date[0]]